In [1]:
"""
import os, shutil, random
from pathlib import Path

BASE = "/kaggle/input/leaf-image-dataset-for-disease-detection-in-bitter/Leaf Image Dataset for Disease Detection in Bitter/Dataset/Dataset"
RAW = os.path.join(BASE, "Raw Data")
AUG = os.path.join(BASE, "Augmented Data")

OUT = "/kaggle/working/working_dataset"

random.seed(42)

def copy_images(src_list, dst_dir):
    os.makedirs(dst_dir, exist_ok=True)
    for f in src_list:
        shutil.copy(f, dst_dir)

for split in ["train","val","test"]:
    shutil.rmtree(os.path.join(OUT,split), ignore_errors=True)

# iterate crops
for crop in os.listdir(RAW):
    crop_path = os.path.join(RAW, crop)

    for disease in os.listdir(crop_path):
        raw_class = os.path.join(crop_path, disease)

        images = [os.path.join(raw_class,i) for i in os.listdir(raw_class)]

        random.shuffle(images)

        n = len(images)
        train_raw = images[:int(n*0.7)]
        val_raw   = images[int(n*0.7):int(n*0.85)]
        test_raw  = images[int(n*0.85):]

        # copy RAW splits
        copy_images(train_raw, f"{OUT}/train/{crop}/{disease}")
        copy_images(val_raw,   f"{OUT}/val/{crop}/{disease}")
        copy_images(test_raw,  f"{OUT}/test/{crop}/{disease}")

        # add AUGMENTED only to TRAIN
        aug_class = os.path.join(AUG, crop, disease)
        if os.path.exists(aug_class):
            aug_imgs = [os.path.join(aug_class,i) for i in os.listdir(aug_class)]
            copy_images(aug_imgs, f"{OUT}/train/{crop}/{disease}")

print("DONE SPLITTING")
"""

'\nimport os, shutil, random\nfrom pathlib import Path\n\nBASE = "/kaggle/input/leaf-image-dataset-for-disease-detection-in-bitter/Leaf Image Dataset for Disease Detection in Bitter/Dataset/Dataset"\nRAW = os.path.join(BASE, "Raw Data")\nAUG = os.path.join(BASE, "Augmented Data")\n\nOUT = "/kaggle/working/working_dataset"\n\nrandom.seed(42)\n\ndef copy_images(src_list, dst_dir):\n    os.makedirs(dst_dir, exist_ok=True)\n    for f in src_list:\n        shutil.copy(f, dst_dir)\n\nfor split in ["train","val","test"]:\n    shutil.rmtree(os.path.join(OUT,split), ignore_errors=True)\n\n# iterate crops\nfor crop in os.listdir(RAW):\n    crop_path = os.path.join(RAW, crop)\n\n    for disease in os.listdir(crop_path):\n        raw_class = os.path.join(crop_path, disease)\n\n        images = [os.path.join(raw_class,i) for i in os.listdir(raw_class)]\n\n        random.shuffle(images)\n\n        n = len(images)\n        train_raw = images[:int(n*0.7)]\n        val_raw   = images[int(n*0.7):int(n*0

In [5]:
import tensorflow as tf

IMG = 224
BATCH = 16
AUTOTUNE = tf.data.AUTOTUNE

def load_dataset(path, shuffle=True):

    ds = tf.keras.utils.image_dataset_from_directory(
        path,
        image_size=(IMG,IMG),
        batch_size=BATCH,
        shuffle=shuffle
    )

    class_names = ds.class_names
    ds = ds.prefetch(AUTOTUNE)

    return ds, class_names


train_crop, CROP_NAMES = load_dataset("/kaggle/working/working_dataset/train")
val_crop, _ = load_dataset("/kaggle/working/working_dataset/val")
test_crop, _ = load_dataset("/kaggle/working/working_dataset/test", shuffle=False)

print("CROPS:", CROP_NAMES)
NUM_CROPS = len(CROP_NAMES)


Found 13839 files belonging to 4 classes.
Found 684 files belonging to 4 classes.
Found 690 files belonging to 4 classes.
CROPS: ['Bitter gourd', 'Okra', 'Pumpkin', 'Ridge gourd']


In [6]:
import os

BASE = "/kaggle/working/working_dataset"

def count_images(path):
    total = 0
    for root,_,files in os.walk(path):
        total += len([f for f in files if f.lower().endswith(('.jpg','.png','.jpeg'))])
    return total

for split in ["train","val","test"]:
    print(f"\n{split.upper()}")
    for crop in os.listdir(os.path.join(BASE,split)):
        for disease in os.listdir(os.path.join(BASE,split,crop)):
            p = os.path.join(BASE,split,crop,disease)
            print(crop,"|",disease,"=",len(os.listdir(p)))

print("\nTOTAL COUNTS")
for s in ["train","val","test"]:
    print(s, count_images(os.path.join(BASE,s)))



TRAIN
Ridge gourd | Downey Mildew = 383
Ridge gourd | Healthy = 355
Okra | Cerospora Leaf Spot = 3078
Okra | Healthy = 3089
Pumpkin | Downey Mildew = 2918
Pumpkin | Healthy = 3049
Bitter gourd | Downey Mildew = 330
Bitter gourd | Anthracnose = 287
Bitter gourd | Healthy = 350

VAL
Ridge gourd | Downey Mildew = 82
Ridge gourd | Healthy = 76
Okra | Cerospora Leaf Spot = 81
Okra | Healthy = 81
Pumpkin | Downey Mildew = 77
Pumpkin | Healthy = 80
Bitter gourd | Downey Mildew = 71
Bitter gourd | Anthracnose = 61
Bitter gourd | Healthy = 75

TEST
Ridge gourd | Downey Mildew = 83
Ridge gourd | Healthy = 77
Okra | Cerospora Leaf Spot = 81
Okra | Healthy = 82
Pumpkin | Downey Mildew = 77
Pumpkin | Healthy = 81
Bitter gourd | Downey Mildew = 71
Bitter gourd | Anthracnose = 62
Bitter gourd | Healthy = 76

TOTAL COUNTS
train 13839
val 684
test 690


In [7]:
!pip -q install keras-tuner
import keras_tuner as kt
from tensorflow import keras
from tensorflow.keras import layers


In [8]:
def build_crop_model(hp):

    base = tf.keras.applications.EfficientNetB0(
        include_top=False,
        weights="imagenet",
        input_shape=(IMG,IMG,3)
    )

    # Tune fine-tuning
    base.trainable = hp.Boolean("fine_tune")

    x = layers.GlobalAveragePooling2D()(base.output)

    # Tune classifier size
    x = layers.Dense(
        hp.Choice("dense_units",[64,128,256]),
        activation="relu")(x)

    x = layers.Dropout(
        hp.Float("dropout",0.2,0.6,step=0.1))(x)

    out = layers.Dense(NUM_CROPS,activation="softmax")(x)

    model = keras.Model(base.input,out)

    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice("lr",[1e-3,3e-4,1e-4])),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    return model


In [9]:
tuner = kt.Hyperband(
    build_crop_model,
    objective="val_accuracy",
    max_epochs=8,
    directory="automl_crop",
    project_name="crop_classifier"
)

tuner.search(train_crop, validation_data=val_crop)


Trial 4 Complete [00h 04m 53s]
val_accuracy: 1.0

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 16m 38s

Search: Running Trial #5

Value             |Best Value So Far |Hyperparameter
False             |True              |fine_tune
128               |256               |dense_units
0.3               |0.4               |dropout
0.0001            |0.0001            |lr
3                 |3                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
1                 |1                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/3
843/865 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.8731 - loss: 0.3645

KeyboardInterrupt: 

In [10]:
crop_model = tuner.get_best_models(1)[0]

history = crop_model.fit(
    train_crop,
    validation_data=val_crop,
    epochs=20
)

crop_model.save("crop_model.keras")


Epoch 1/20


/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 432 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


865/865 ━━━━━━━━━━━━━━━━━━━━ 166s 118ms/step - accuracy: 0.9985 - loss: 0.0053 - val_accuracy: 1.0000 - val_loss: 1.2221e-04
Epoch 2/20
865/865 ━━━━━━━━━━━━━━━━━━━━ 60s 70ms/step - accuracy: 0.9996 - loss: 0.0020 - val_accuracy: 1.0000 - val_loss: 8.9335e-05
Epoch 3/20
865/865 ━━━━━━━━━━━━━━━━━━━━ 61s 71ms/step - accuracy: 0.9995 - loss: 0.0023 - val_accuracy: 1.0000 - val_loss: 6.7171e-06
Epoch 4/20
865/865 ━━━━━━━━━━━━━━━━━━━━ 61s 71ms/step - accuracy: 0.9996 - loss: 0.0012 - val_accuracy: 1.0000 - val_loss: 0.0025
Epoch 5/20
865/865 ━━━━━━━━━━━━━━━━━━━━ 60s 70ms/step - accuracy: 0.9982 - loss: 0.0053 - val_accuracy: 1.0000 - val_loss: 8.3184e-06
Epoch 6/20
865/865 ━━━━━━━━━━━━━━━━━━━━ 60s 70ms/step - accuracy: 1.0000 - loss: 1.7092e-04 - val_accuracy: 1.0000 - val_loss: 5.4604e-06
Epoch 7/20
865/865 ━━━━━━━━━━━━━━━━━━━━ 61s 70ms/step - accuracy: 1.0000 - loss: 4.1057e-05 - val_accuracy: 1.0000 - val_loss: 1.5716e-06
Epoch 8/20
865/865 ━━━━━━━━━━━━━━━━━━━━ 60s 70ms/step - accuracy: 0

In [11]:
loss, acc = crop_model.evaluate(test_crop)
print("FINAL REAL-WORLD ACCURACY:", acc)


44/44 ━━━━━━━━━━━━━━━━━━━━ 8s 182ms/step - accuracy: 1.0000 - loss: 3.8961e-06
FINAL REAL-WORLD ACCURACY: 1.0


In [13]:
def load_disease_ds(crop):

    train = tf.keras.utils.image_dataset_from_directory(
        f"/kaggle/working/working_dataset/train/{crop}",
        image_size=(IMG,IMG),
        batch_size=BATCH
    )
    class_names = train.class_names

    val = tf.keras.utils.image_dataset_from_directory(
        f"/kaggle/working/working_dataset/val/{crop}",
        image_size=(IMG,IMG),
        batch_size=BATCH
    )

    test = tf.keras.utils.image_dataset_from_directory(
        f"/kaggle/working/working_dataset/test/{crop}",
        image_size=(IMG,IMG),
        batch_size=BATCH,
        shuffle=False
    )

    # apply performance optimization AFTER metadata extraction
    train = train.prefetch(AUTOTUNE)
    val   = val.prefetch(AUTOTUNE)
    test  = test.prefetch(AUTOTUNE)

    return train, val, test, class_names


In [ ]:
from tensorflow.keras import layers, Model
import tensorflow as tf

def build_disease_model(backbone_name, num_classes):

    if backbone_name == "efficientnet":
        base = tf.keras.applications.EfficientNetB0(
            include_top=False,
            weights="imagenet",
            input_shape=(IMG,IMG,3)
        )

    elif backbone_name == "mobilenet":
        base = tf.keras.applications.MobileNetV3Large(
            include_top=False,
            weights="imagenet",
            input_shape=(IMG,IMG,3)
        )

    else:
        raise ValueError("Unknown backbone")

    x = layers.GlobalAveragePooling2D()(base.output)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dropout(0.5)(x)

    logits = layers.Dense(num_classes)(x)

    model = Model(base.input, logits)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-4),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"]
    )

    return model


In [14]:
from tensorflow.keras import layers, Model
import tensorflow as tf

def build_disease_model(backbone_name, num_classes):

    if backbone_name == "efficientnet":
        base = tf.keras.applications.EfficientNetB0(
            include_top=False,
            weights="imagenet",
            input_shape=(IMG,IMG,3)
        )

    elif backbone_name == "mobilenet":
        base = tf.keras.applications.MobileNetV3Large(
            include_top=False,
            weights="imagenet",
            input_shape=(IMG,IMG,3)
        )

    else:
        raise ValueError("Unknown backbone")

    # Freeze first (important for small agricultural dataset)
    base.trainable = False

    x = layers.GlobalAveragePooling2D()(base.output)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dropout(0.5)(x)

    logits = layers.Dense(num_classes)(x)

    model = Model(base.input, logits)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-4),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"]
    )

    return model


In [15]:
results = {}

for crop in CROP_NAMES:

    print("\n==============================")
    print("CROP:", crop)
    print("==============================")

    train, val, test, classes = load_disease_ds(crop)
    print("Classes:", classes)

    results[crop] = {}

    for backbone in ["efficientnet", "mobilenet"]:

        print("\nTraining", backbone)

        model = build_disease_model(backbone, len(classes))

        history = model.fit(
            train,
            validation_data=val,
            epochs=12,
            verbose=1
        )

        loss, acc = model.evaluate(test, verbose=0)

        print("Test Accuracy:", acc)

        results[crop][backbone] = acc

        model.save(f"{crop}_{backbone}.keras")



CROP: Bitter gourd
Found 967 files belonging to 3 classes.
Found 207 files belonging to 3 classes.
Found 209 files belonging to 3 classes.
Classes: ['Anthracnose', 'Downey Mildew', 'Healthy']

Training efficientnet
Epoch 1/12
61/61 ━━━━━━━━━━━━━━━━━━━━ 42s 388ms/step - accuracy: 0.5010 - loss: 1.2448 - val_accuracy: 0.9130 - val_loss: 0.6308
Epoch 2/12
61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.8022 - loss: 0.5123 - val_accuracy: 0.9179 - val_loss: 0.4226
Epoch 3/12
61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.8289 - loss: 0.4555 - val_accuracy: 0.9275 - val_loss: 0.2971
Epoch 4/12
61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.8772 - loss: 0.3522 - val_accuracy: 0.9372 - val_loss: 0.2268
Epoch 5/12
61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.8863 - loss: 0.3135 - val_accuracy: 0.9517 - val_loss: 0.1761
Epoch 6/12
61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.8792 - loss: 0.2925 - val_accuracy: 0.9517 - val_loss: 0.1537
Epoch 7/12
61/61 ━━━

In [16]:
print("\n\nFINAL RESULTS")
print("=================================")

for crop in results:
    print(f"\n{crop}")
    for model_name in results[crop]:
        print(f"{model_name}: {results[crop][model_name]:.4f}")




FINAL RESULTS

Bitter gourd
efficientnet: 0.9761
mobilenet: 0.9713

Okra
efficientnet: 1.0000
mobilenet: 0.9939

Pumpkin
efficientnet: 0.9937
mobilenet: 0.9873

Ridge gourd
efficientnet: 1.0000
mobilenet: 1.0000


In [17]:
import random
import tensorflow as tf

def cross_crop_test():

    for source_crop in CROP_NAMES:

        _,_,test,_ = load_disease_ds(source_crop)
        img,label = next(iter(test.take(1)))

        wrong_crop = random.choice([c for c in CROP_NAMES if c != source_crop])
        wrong_model = tf.keras.models.load_model(f"{wrong_crop}_efficientnet.keras")

        pred = wrong_model(img, training=False)
        conf = tf.nn.softmax(pred)[0].numpy().max()

        print(f"\nActual crop: {source_crop}")
        print(f"Tested on model: {wrong_crop}")
        print(f"Confidence:", conf)

cross_crop_test()


Found 967 files belonging to 3 classes.
Found 207 files belonging to 3 classes.
Found 209 files belonging to 3 classes.

Actual crop: Bitter gourd
Tested on model: Pumpkin
Confidence: 1.0
Found 6167 files belonging to 2 classes.
Found 162 files belonging to 2 classes.
Found 163 files belonging to 2 classes.

Actual crop: Okra
Tested on model: Pumpkin
Confidence: 0.96768314
Found 5967 files belonging to 2 classes.
Found 157 files belonging to 2 classes.
Found 158 files belonging to 2 classes.

Actual crop: Pumpkin
Tested on model: Okra
Confidence: 0.99999726
Found 738 files belonging to 2 classes.
Found 158 files belonging to 2 classes.
Found 160 files belonging to 2 classes.

Actual crop: Ridge gourd
Tested on model: Bitter gourd
Confidence: 0.99566615


In [18]:
import tensorflow as tf
import numpy as np

def energy_score(logits):
    return -tf.reduce_logsumexp(logits, axis=1)


In [19]:
def collect_energy(model, dataset):

    energies = []

    for img,label in dataset:
        logits = model(img, training=False)
        e = energy_score(logits)
        energies.extend(e.numpy())

    return np.array(energies)


In [20]:
thresholds = {}

for crop in CROP_NAMES:

    print("\nCalibrating:",crop)

    _, val_ds, _, _ = load_disease_ds(crop)
    model = tf.keras.models.load_model(f"{crop}_efficientnet.keras")

    energy_vals = collect_energy(model, val_ds)

    threshold = np.percentile(energy_vals, 95)  # allow 95% known acceptance
    thresholds[crop] = threshold

    print("Threshold:",threshold)



Calibrating: Bitter gourd
Found 967 files belonging to 3 classes.
Found 207 files belonging to 3 classes.
Found 209 files belonging to 3 classes.
Threshold: -2.0138755

Calibrating: Okra
Found 6167 files belonging to 2 classes.
Found 162 files belonging to 2 classes.
Found 163 files belonging to 2 classes.
Threshold: -2.0678878

Calibrating: Pumpkin
Found 5967 files belonging to 2 classes.
Found 157 files belonging to 2 classes.
Found 158 files belonging to 2 classes.
Threshold: -1.4015038

Calibrating: Ridge gourd
Found 738 files belonging to 2 classes.
Found 158 files belonging to 2 classes.
Found 160 files belonging to 2 classes.
Threshold: -2.0293026


In [21]:
def predict_safe(image):

    crop_probs = crop_model.predict(image)
    crop_id = crop_probs.argmax()
    crop = CROP_NAMES[crop_id]

    model = tf.keras.models.load_model(f"{crop}_efficientnet.keras")

    logits = model(image, training=False)
    e = energy_score(logits).numpy()[0]

    if e > thresholds[crop]:
        return crop, "Unknown Disease"

    disease_id = tf.argmax(logits,axis=1).numpy()[0]
    return crop, disease_id


In [22]:
def cross_crop_safe_test():

    for source_crop in CROP_NAMES:

        _,_,test,_ = load_disease_ds(source_crop)
        img,label = next(iter(test.take(1)))

        crop, disease = predict_safe(img)

        print("\nActual:",source_crop)
        print("Prediction:",crop,disease)

cross_crop_safe_test()


Found 967 files belonging to 3 classes.
Found 207 files belonging to 3 classes.
Found 209 files belonging to 3 classes.
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step

Actual: Bitter gourd
Prediction: Bitter gourd 1
Found 6167 files belonging to 2 classes.
Found 162 files belonging to 2 classes.
Found 163 files belonging to 2 classes.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step

Actual: Okra
Prediction: Okra 0
Found 5967 files belonging to 2 classes.
Found 157 files belonging to 2 classes.
Found 158 files belonging to 2 classes.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step

Actual: Pumpkin
Prediction: Pumpkin 0
Found 738 files belonging to 2 classes.
Found 158 files belonging to 2 classes.
Found 160 files belonging to 2 classes.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step

Actual: Ridge gourd
Prediction: Ridge gourd 0


In [23]:
def collect_energy_known_unknown(crop):

    model = tf.keras.models.load_model(f"{crop}_efficientnet.keras")

    # known samples
    _, val_known, _, _ = load_disease_ds(crop)
    known_energy = collect_energy(model, val_known)

    # unknown samples (other crops)
    unknown_energy = []

    for other in CROP_NAMES:
        if other == crop:
            continue

        _, val_other, _, _ = load_disease_ds(other)

        for img,_ in val_other:
            logits = model(img, training=False)
            e = energy_score(logits)
            unknown_energy.extend(e.numpy())

    return np.array(known_energy), np.array(unknown_energy)


In [24]:
thresholds = {}

for crop in CROP_NAMES:

    print("\nCalibrating:",crop)

    known, unknown = collect_energy_known_unknown(crop)

    print("Known avg:",known.mean())
    print("Unknown avg:",unknown.mean())

    # midpoint between distributions
    threshold = (known.mean() + unknown.mean()) / 2
    thresholds[crop] = threshold

    print("Chosen threshold:",threshold)



Calibrating: Bitter gourd
Found 967 files belonging to 3 classes.
Found 207 files belonging to 3 classes.
Found 209 files belonging to 3 classes.
Found 6167 files belonging to 2 classes.
Found 162 files belonging to 2 classes.
Found 163 files belonging to 2 classes.
Found 5967 files belonging to 2 classes.
Found 157 files belonging to 2 classes.
Found 158 files belonging to 2 classes.
Found 738 files belonging to 2 classes.
Found 158 files belonging to 2 classes.
Found 160 files belonging to 2 classes.
Known avg: -4.3829503
Unknown avg: -3.1536016
Chosen threshold: -3.768276

Calibrating: Okra
Found 6167 files belonging to 2 classes.
Found 162 files belonging to 2 classes.
Found 163 files belonging to 2 classes.
Found 967 files belonging to 3 classes.
Found 207 files belonging to 3 classes.
Found 209 files belonging to 3 classes.
Found 5967 files belonging to 2 classes.
Found 157 files belonging to 2 classes.
Found 158 files belonging to 2 classes.
Found 738 files belonging to 2 class

In [25]:
def predict_safe(image):

    crop_probs = crop_model.predict(image)
    crop_id = crop_probs.argmax()
    crop = CROP_NAMES[crop_id]

    model = tf.keras.models.load_model(f"{crop}_efficientnet.keras")
    classes = load_disease_ds(crop)[3]

    logits = model(image, training=False)
    e = energy_score(logits).numpy()[0]

    if e > thresholds[crop]:
        return crop, "Unknown Disease"

    disease_id = tf.argmax(logits,axis=1).numpy()[0]
    return crop, classes[disease_id]


In [26]:
cross_crop_safe_test()


Found 967 files belonging to 3 classes.
Found 207 files belonging to 3 classes.
Found 209 files belonging to 3 classes.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Found 967 files belonging to 3 classes.
Found 207 files belonging to 3 classes.
Found 209 files belonging to 3 classes.

Actual: Bitter gourd
Prediction: Bitter gourd Downey Mildew
Found 6167 files belonging to 2 classes.
Found 162 files belonging to 2 classes.
Found 163 files belonging to 2 classes.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Found 6167 files belonging to 2 classes.
Found 162 files belonging to 2 classes.
Found 163 files belonging to 2 classes.

Actual: Okra
Prediction: Okra Cerospora Leaf Spot
Found 5967 files belonging to 2 classes.
Found 157 files belonging to 2 classes.
Found 158 files belonging to 2 classes.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Found 5967 files belonging to 2 classes.
Found 157 files belonging to 2 classes.
Found 158 files belonging to 2 classes.

Actual: Pumpkin
Prediction: Pumpkin Downey Mildew

In [27]:
from tensorflow.keras import Model

def get_feature_extractor(model):
    return Model(model.input, model.layers[-3].output)


In [28]:
import numpy as np

def compute_prototypes(model, dataset, num_classes):

    extractor = get_feature_extractor(model)

    feats = [[] for _ in range(num_classes)]

    for img, label in dataset:
        emb = extractor(img, training=False).numpy()
        for i,l in enumerate(label.numpy()):
            feats[l].append(emb[i])

    prototypes = [np.mean(f, axis=0) for f in feats]
    return np.array(prototypes)


In [29]:
from scipy.spatial.distance import cdist

proto_bank = {}
dist_thresholds = {}

for crop in CROP_NAMES:

    print("\nCalibrating:", crop)

    train, val, _, classes = load_disease_ds(crop)
    model = tf.keras.models.load_model(f"{crop}_efficientnet.keras")

    prototypes = compute_prototypes(model, train, len(classes))
    proto_bank[crop] = prototypes

    extractor = get_feature_extractor(model)

    dists = []

    for img,_ in val:
        emb = extractor(img, training=False).numpy()
        dist = cdist(emb, prototypes).min(axis=1)
        dists.extend(dist)

    threshold = np.percentile(dists, 95)
    dist_thresholds[crop] = threshold

    print("Threshold:", threshold)



Calibrating: Bitter gourd
Found 967 files belonging to 3 classes.
Found 207 files belonging to 3 classes.
Found 209 files belonging to 3 classes.
Threshold: 10.934633910785847

Calibrating: Okra
Found 6167 files belonging to 2 classes.
Found 162 files belonging to 2 classes.
Found 163 files belonging to 2 classes.
Threshold: 14.641745422117197

Calibrating: Pumpkin
Found 5967 files belonging to 2 classes.
Found 157 files belonging to 2 classes.
Found 158 files belonging to 2 classes.
Threshold: 16.85634665596395

Calibrating: Ridge gourd
Found 738 files belonging to 2 classes.
Found 158 files belonging to 2 classes.
Found 160 files belonging to 2 classes.
Threshold: 9.920366590954618


In [30]:
def predict_safe(image):

    crop_probs = crop_model.predict(image)
    crop_id = crop_probs.argmax()
    crop = CROP_NAMES[crop_id]

    model = tf.keras.models.load_model(f"{crop}_efficientnet.keras")
    extractor = get_feature_extractor(model)
    prototypes = proto_bank[crop]
    classes = load_disease_ds(crop)[3]

    emb = extractor(image, training=False).numpy()
    dist = cdist(emb, prototypes).min()

    if dist > dist_thresholds[crop]:
        return crop, "Unknown Disease"

    logits = model(image, training=False)
    disease_id = tf.argmax(logits,axis=1).numpy()[0]

    return crop, classes[disease_id]


In [31]:
cross_crop_safe_test()


Found 967 files belonging to 3 classes.
Found 207 files belonging to 3 classes.
Found 209 files belonging to 3 classes.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Found 967 files belonging to 3 classes.
Found 207 files belonging to 3 classes.
Found 209 files belonging to 3 classes.

Actual: Bitter gourd
Prediction: Bitter gourd Downey Mildew
Found 6167 files belonging to 2 classes.
Found 162 files belonging to 2 classes.
Found 163 files belonging to 2 classes.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Found 6167 files belonging to 2 classes.
Found 162 files belonging to 2 classes.
Found 163 files belonging to 2 classes.

Actual: Okra
Prediction: Okra Cerospora Leaf Spot
Found 5967 files belonging to 2 classes.
Found 157 files belonging to 2 classes.
Found 158 files belonging to 2 classes.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Found 5967 files belonging to 2 classes.
Found 157 files belonging to 2 classes.
Found 158 files belonging to 2 classes.

Actual: Pumpkin
Prediction: Pumpkin Downey Mildew